<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03B_%7C_Repaso_II_sobre_%C3%81rboles_Decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 REPASO II: Árboles de Decisión
Repaso sobre cómo utilizar árboles de decisión y cómo realizar la preparación previa de los datos.

Notebook por [Javier Blanco Cordero](https://www.linkedin.com/in/javier-blanco-cordero-71373656/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1jRg7Dk2y_2_fxnC_Jpj5aWcqgW9t1KAd7izdmWzv9Sk/edit?usp=sharing)
*   [Enlace a este notebook](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_03_Arboles/03A_%7C_Repaso_I_sobre_%C3%81rboles_Decisi%C3%B3n.ipynb)




## 0101 Qué es un árbol de decisión?
Un tipo de algoritmo de aprendizaje supervisado que se basa en realizar particiones recursivas a partir de distintos niveles de las variables disponibles.

## 0102 Import
Importamos todas las librerías necesarias para este análisis ([¿No sabes lo que es una librería de Python?](https://www.quora.com/What-is-a-Python-library-and-what-can-I-use-it-for)): pandas, numpy, seaborn, matplotlib.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 0103 Carga el dataset del Titanic
Recordais el dataset del Titanic?

Vamos a hacer un árbol de decisión que prediga si alguien fallece o no, pero esta vez el dataset no está ya limpio y preparado. Este mismo ejemplo lo resolvisteis en el [notebook 2C](https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/Arboles/Clase_02_Arboles/02_C_%C3%81rboles_Decisi%C3%B3n_sin_Overfitting_Resuelto.ipynb#scrollTo=EDiMGl7xOm6o).

Para ello utilizaremos el dataset original. Este es el link al archivo raw: https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv.

Importa los datos en un dataframe llamado **dataframe df_titanic**.

In [ ]:
# Url archivo raw
url = 'https://raw.githubusercontent.com/JotaBlanco/TheValley/main/Data/titanic.csv'

# Importa csv
df_titanic = pd.read_csv(url)

# Visualización primeras filas
df_titanic.head()

# 02 EDA
Realizaremos un pequeño análisis exploratorio visual para familiarizarnos con el dataset. 

Recuerda que puedes encontrar mis clases sobre análisis exploratorio [aquí](https://github.com/JotaBlanco/TheValley/tree/main/EDA/).

In [ ]:
df_titanic.info()

In [ ]:
df_titanic.describe()

In [ ]:
# Visualización coeficientes Pearson
plt.figure(figsize=(8,7))
sns.heatmap(np.round(df_titanic.corr(),2), 
            vmin=-1, vmax=1, 
            annot=True, cmap="coolwarm")
plt.show()

# 03 Preparación de los datos
Primero prepararemos el dataset para que admita un entrenamiento de un árbol de decisión.

## 0301 Creación de variables
Primero, vamos a crear una variable nueva:

In [ ]:
# Creamos una nueva variable Título (es opcional, pero podría ayudar al modelo)
df_titanic['Title'] = [name.split(",")[1].split(".")[0][1:] for name in df_titanic['Name']]
df_titanic.head(2)

## 0302 Seleccionamos variables
Nos quedamos con las variables que utilizaremos. 

Descartamos ciertas columnas como el nombre, o la cabina: tienen muchas posibles categorías, y no deberían ser buenas para predecir el target sobre datos aún no vistos.

In [ ]:
df_titanic.columns

In [ ]:
# Columnas con las que nos quedamos
cols = ['Survived', 
        'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']

# De ellas, cuáles son categóricas
cat_cols = ['Sex', 'Embarked', 'Title']

# Visualizamos las columnas con las que nos hemos quedado
df_titanic[cols].head()

## 0303 Limpieza de nulos
Los árboles de decisión no admiten nulos, por lo que tenemos que hacer algo con ellos (eliminarlos o rellenarlos con ciertos valores).

In [ ]:
# Rellenamos los nulos de las variables numéricas (edad)
df_titanic[cols].isna().sum()

In [ ]:
# Rellenamos los nulos de la variable edad de una forma avanzada:
# calculando la media para cada título

filtro_edad_nula = df_titanic['Age'].isna()
filtro_edad_no_nula = df_titanic['Age'].notnull()

# Rellenamos los nulos en función del título 
for titulo in df_titanic['Title'].unique():
  filtro_titulo = df_titanic['Title'] == titulo

  if len(df_titanic[((filtro_titulo) & (filtro_edad_no_nula))]) > 2:
    print(titulo)
    df_titanic.loc[((filtro_titulo)&(filtro_edad_nula)), 'Age'] = df_titanic.loc[((filtro_titulo)&(filtro_edad_no_nula)), 'Age'].median()
  
# Rellenamos los que sigan siendo nulos (títulos solo presentes entre gente sin la edad informada)
df_titanic['Age'] = df_titanic['Age'].fillna(df_titanic['Age'].median())

In [ ]:
# Visualización de nulos
df_titanic[cols].isna().sum()

In [ ]:
# Eliminamos los nulos de la variable Embarked
print(len(df_titanic))
df_titanic = df_titanic.dropna(subset=['Embarked'])
print(len(df_titanic))

## 0304 Dumificación variables categóricas
Por último, dumificamos las variables categóricas para convertirlas en numéricas:

In [ ]:
# Dumificamos variables categóricas
df_titanic_i = pd.get_dummies(df_titanic[cols], 
                              prefix_sep='_',
                              drop_first=True, 
                              columns=cat_cols)
df_titanic_i.head()

In [ ]:
df_titanic_i.isna().sum()

# 04 Train - Test split
Una de las principales medidas contra el sobreajuste consiste en reservar un set de datos para testear el modelo.

Utilizaremos una partición del 25%

In [ ]:
# Generamos las matrices X e y
X = df_titanic_i.drop('Survived',axis=1)
y = df_titanic_i['Survived']

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 05 Construcción de árboles de decisión 
Utilizaremos la librería [scikit learn](https://scikit-learn.org/stable/), la libería básica de referencia para machine learning, para entrenar varios árboles de decisión y comprobar su equilibrio bias-variance.

## 0501 Ejemplo con un modelo
Ejemplo de cómo entrenar y evaluar un modelo.

In [ ]:
# Inicializo un árbol con 10 de profundidad
modelo = tree.DecisionTreeClassifier(max_depth=10)

In [ ]:
# Entreno el árbol con el set de entrenamiento
modelo = modelo.fit(X=X_train, y=y_train)

In [ ]:
# Uso el árbol para predecir sobre el dataset de entrenamiento y de prueba
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

In [ ]:
# Cómo de buenas son las predicciones?
from sklearn.metrics import accuracy_score

ac_train = round(accuracy_score(y_train, y_pred_train), 4)
ac_test = round(accuracy_score(y_test, y_pred_test), 4)

print('Precisión en set de entrenamiento :', ac_train)
print('Precisión en set de test :', ac_test)
print('Degradación: ', round((ac_train-ac_test)/ac_train*100,2), '%')

## 0502 Generamos función
Que dado un modelo inicializado, lo entrena y lo evalúa:

In [ ]:
def entrenar_modelo_y_predecir_classificacion(modelo):
  # Entreno el árbol con el set de entrenamiento
  modelo = modelo.fit(X=X_train, y=y_train)
  # Uso el árbol para predecir sobre el dataset de entrenamiento
  y_pred_train = modelo.predict(X_train)
  # Uso el árbol para predecir sobre el dataset de test
  y_pred_test = modelo.predict(X_test)
  # Cómo de buena es la predicción?
  ac_train = round(accuracy_score(y_train, y_pred_train), 4)
  print('Precisión en set de entrenamiento :', ac_train)
  ac_test = round(accuracy_score(y_test, y_pred_test), 4)
  print('Precisión en set de test :', ac_test)
  print('Degradación: ', round((ac_train-ac_test)/ac_train*100,2), '%')

In [ ]:
# Inicializo un árbol con 10 de profundidad
modelo = tree.DecisionTreeClassifier(max_depth=10)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

In [ ]:
# Inicializo un árbol con 15 de profundidad
modelo = tree.DecisionTreeClassifier(max_depth=15)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

## 0503 EJERCICIO: Equilibrio Bias - Variance
Tienes varias medidas para controlar el equilibrio bias-variance:


*   max_depth determina la profundidad de tu árbol
*   min_samples_split tamaño muestral mínimo en un nodo para permitir una partición
*   min_samples_leaf = 10 tamaño muestral mínimo en un nodo terminal (en una hoja del árbol)
*   min_impurity_decrease mínimo descenso de impuridad que tiene que provocar cada partición

Juega con estos valores para encontrar el mejor modelo posible.



In [ ]:
modelo = tree.DecisionTreeClassifier(max_depth=15, 
                                     min_samples_split = 20,
                                     min_samples_leaf = 5)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)

In [ ]:
modelo = tree.DecisionTreeClassifier(max_depth=20, 
                                     min_samples_split = 20,
                                     min_samples_leaf = 5, 
                                     min_impurity_decrease = 0.003)
# Entrenamos y predecimos con dicho modelo
entrenar_modelo_y_predecir_classificacion(modelo)